## Financial Sentiment Analysis

##### CS6120 - Natural Language Processing

##### Anna Brunkhorst, Nader Lobandi, Ashish Magadum

#### Importing Necessary Libraries

In [10]:
# Importing necessary libraries
!pip install datasets
from datasets import load_dataset
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string

# Download NLTK resources
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\abrun\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abrun\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\abrun\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


True

#### Load Datasets

In [11]:
# Load Yahoo-Finance-News-Sentences dataset from Huggingface
ugursa_ds = load_dataset("ugursa/Yahoo-Finance-News-Sentences")

# Convert to Pandas DF for easier manipulation
ugursa_df = pd.DataFrame(ugursa_ds['train'])

# Check some data
ugursa_df.head()

,label,text
0,0,Chinese-owned companies are aggressively expan...
1,0,Chinese cobalt producers have seemed unfazed b...
2,0,"China's CMOC Group, which boosted its cobalt o..."
3,0,CMOC is due to lift its market share of the gl...
4,1,Its Kisanfu mine in Democratic Republic of Con...


#### Preprocess Data

In [12]:
# Function to preprocess text
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize text
    tokens = word_tokenize(text)
    # Remove stopwords
    tokens = [word for word in tokens if not word in stopwords.words('english')]
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return " ".join(tokens)

# Clone DF for preprocessing
ugursa_df_pre = ugursa_df.copy()

# Apply preprocessing to each text entry
ugursa_df_pre['text'] = ugursa_df['text'].apply(preprocess_text)

# Check some data
ugursa_df_pre.head()

,label,text
0,0,chineseowned company aggressively expanding co...
1,0,chinese cobalt producer seemed unfazed oversup...
2,0,china cmoc group boosted cobalt output 144 fir...
3,0,cmoc due lift market share global mined cobalt...
4,1,kisanfu mine democratic republic congo drc par...
